In [ ]:
import numpy as np

from keras.preprocessing import sequence
import keras.models as km
import keras.layers as kl
import keras.constraints as kc
from keras.datasets import imdb

import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as ipw

import data_preparation as mdp
from mercari_config import MercariConfig
from timer import Timer

In [ ]:
num_words = MercariConfig.MAX_WORDS_FROM_INDEX + MercariConfig.INDICATOR_WORDS_COUNT
max_len = MercariConfig.MAX_WORDS_ITEM_DESC + 1
batch_size = 32

In [ ]:
with Timer():
    train_word_seq = mdp.load_data(MercariConfig.TRAINING_ITEM_DESC_W2I_FILE)
    X_train = train_word_seq.as_matrix()

In [ ]:
with Timer():
    train_data = mdp.load_data(MercariConfig.TRAINING_SET_PREP_FILE)
    y_train = train_data['price'].as_matrix()

In [ ]:
with Timer():
    X_train = sequence.pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
#x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len, padding='post', truncating='post')

#y_train = np.array(y_train)
#y_test = np.array(y_test)

In [ ]:
X_train

In [ ]:
sequence_input = kl.Input(shape=(maxlen,))

sequence_embeddings = kl.Embedding(num_words + 3, 32)
lstm_layer = kl.CuDNNLSTM(32)
clf_layer = kl.Dense(1, activation='sigmoid')

output = sequence_embeddings(sequence_input)
output = kl.SpatialDropout1D(0.5)(output)
output = lstm_layer(output)
output = kl.Dropout(0.5)(output)
output = clf_layer(output)

simple_model = km.Model(inputs=sequence_input, outputs=output)

In [ ]:
simple_model.summary()

In [ ]:
simple_model.compile('adadelta', 'binary_crossentropy', metrics=['accuracy'])

history_simple = simple_model.fit(
    x_train_pad, y_train,
    batch_size=batch_size,
    epochs=10,
    shuffle=True,
    validation_data=[x_test_pad, y_test])